<a href="https://colab.research.google.com/github/beingnilesh07/LLM_Practicals/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade accelerate
# !pip uninstal -y transformers accelerate
# !pip install transformers accelerate

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [ ]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset, load_from_disk
import torch
import pandas as pd
import evaluate #updated from laod_metric to evaluate

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
#Download the punkt tokenizer for sentence splitting
nltk.download("punkt")
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model_ckpt='google/pegasus-cnn_dailymail'
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
dataset_samsum=load_dataset('knkarthick/samsum')

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

This code will search for datasets on the Hugging Face Hub that contain 'samsum' in their name. If the previous error was due to a typo, this should help you find the correct name.

In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [ ]:
dataset_samsum['train']['dialogue'][1]

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [ ]:
dataset_samsum['train'][1]['summary']

'Olivia and Olivier are voting for liberals in this election. '

In [ ]:
split_length=[len(dataset_samsum[split]) for split in dataset_samsum]

In [ ]:
print(f'split lengths:{split_length}')
print(f"Features:{dataset_samsum['train'].column_names}")


split lengths:[14731, 818, 819]
Features:['id', 'dialogue', 'summary']


In [ ]:
print(dataset_samsum['test'][1]['dialogue'])
print('\nsummary:')
print(dataset_samsum['test'][1]['summary'])

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

summary:
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings=tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings=tokenizer(example_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids':input_encodings['input_ids'],
        'attention_mask':input_encodings['attention_mask'],
        'labels':target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [ ]:
dataset_samsum_pt['train']['input_ids'][1]

[18038,
 151,
 2632,
 127,
 119,
 6228,
 118,
 115,
 136,
 2974,
 152,
 10463,
 151,
 35884,
 130,
 329,
 107,
 18038,
 151,
 2587,
 314,
 1242,
 10463,
 151,
 1509,
 1]

In [ ]:
dataset_samsum_pt['train']['attention_mask'][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
#Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer



In [ ]:
trainer_args=TrainingArguments(
  output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    do_eval=True, eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
# !pip install --upgrade transformers accelerate

In [ ]:
trainer=Trainer(model=model_pegasus, args=trainer_args,
  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt['test'],
    eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kabirp618 (kabirp618-north-maharashtra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.832300
20,3.157300
30,3.177300
40,2.903300
50,2.737800


TrainOutput(global_step=52, training_loss=2.979437919763418, metrics={'train_runtime': 446.5927, 'train_samples_per_second': 1.834, 'train_steps_per_second': 0.116, 'total_flos': 314017624350720.0, 'train_loss': 2.979437919763418, 'epoch': 1.0})

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                      length_penalty=0.8, num_beams=8, max_length=128)
        #parameter for lenth penalty ensures the model does not generate sequences that are too long.

        #finally we decodethe generated text,
        #replace the token and add the decoded texts with references to metric
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #finally compute and return the rouge scores
        score=metric.compute()
        return score

In [ ]:
!pip install rouge_score --upgrade
!pip install evaluate --upgrade
import evaluate
rouge_names=['rouge1', 'rouge2', 'rouge1', 'rougeL', 'rougeLsum']
rouge_metric= evaluate.load('rouge')

In [ ]:
#!pip install rouge_score

In [ ]:
#calculate metric on test dataset
score=calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
)

#update this to directly access the score since 'mid ' and 'fmeasure' attributes dont exits
rouge_dict=dict((rn, score[rn]) for rn in rouge_names)

#create dataframe to display results
pd.DataFrame(rouge_dict, index=[f'pegasus'])

  0%|          | 0/5 [00:09<?, ?it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023302,0.0,0.023302,0.023302


In [ ]:
#save model
model_pegasus.save_pretrained('pegasus-samsum-model')

In [ ]:
#save tokenizer
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
# load
tokenizer=AutoTokenizer.from_pretrained('/content/tokenizer')


In [ ]:
gen_kwargs={'length_penalty':0.8, 'num_beams':8, 'max_length':128}

In [ ]:
sampling_text=dataset_samsum['test'][0]['dialogue']

reference= dataset_samsum['test'][0]['summary']

In [ ]:
pipe=pipeline('summarization', model='pegasus-samsum-model', tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
print('Dialogue:')
print(sampling_text)
print('\nReference Summary:')
print(reference)
print('\nModel Summary:')
print(pipe(sampling_text, **gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
